In [1]:
import json
import numpy as np
import math
import shapely
from shapely.geometry import Point, Polygon, LineString
from scipy.spatial import KDTree
from shapely.affinity import rotate
from shapely.ops import unary_union, nearest_points
import random
from tqdm.auto import tqdm
import vtk
import vtkmodules.util.numpy_support as vtk_np
from vtk.util.numpy_support import vtk_to_numpy
from scipy.ndimage import generate_binary_structure, binary_dilation, binary_erosion
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import copy
from PIL import Image, ImageDraw, ImageFont

**Запись итогового json с геометрией**

В качестве входных данных используется json сгенерированный в Grasshopper с планарной информацией о сцене.

In [2]:
planar_data_path = ...

# Принимается исходный json с планарной геметрией
with open(planar_data_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Проверка равенства количество сцен домов и полигонов участков
q_len = len(data['ins']) if len(data['ins']) == len(data['out']) else None

In [3]:
# Инициализация диапазона высот для будующих домов (мм)
h_min = 10000
h_max = 80000

In [4]:
# Возвращает сумму измерений по каждому из направлений (орто)
def get_direction(in_polys):
    x_total = 0.0
    y_total = 0.0
    all_coords = []
    for poly_coords in in_polys:
        all_coords.extend(poly_coords)
        for i in range(len(poly_coords) - 1):
            x1, y1 = poly_coords[i]
            x2, y2 = poly_coords[i + 1]
            dx = abs(x2 - x1)
            dy = abs(y2 - y1)
            if dx > 0 and dy == 0:
                x_total += dx
            elif dy > 0 and dx == 0:
                y_total += dy
    return x_total, y_total

# Делает центроид началом отсчёта, максимальную кооринату единицей, доминируемую ось y
def normalize(in_polys, out_poly):
    sin_polys = [Polygon(coords) for coords in in_polys]
    merged = unary_union(sin_polys)
    centroid = merged.centroid
    cx, cy = centroid.x, centroid.y
    shifted_coords = [[[x - cx, y - cy] for x, y in poly] for poly in in_polys]
    shifted_coords_out = [[x - cx, y - cy] for x, y in out_poly]
    flat_coords = np.array([pt for poly in shifted_coords for pt in poly])
    max_coord = np.abs(flat_coords).max()
    normalized_coords = [[[x / max_coord, y / max_coord] for x, y in poly] for poly in shifted_coords]
    normalized_coords_out = [[x / max_coord, y / max_coord] for x, y in shifted_coords_out]
    x_total, y_total = get_direction(normalized_coords)
    if x_total > y_total:
        normalized_coords = [[[y, -x] for x, y in poly] for poly in normalized_coords]
        normalized_coords_out = [[y, -x] for x, y in normalized_coords_out]
    return normalized_coords, normalized_coords_out, 1/max_coord

# Отражает координаты и проверяет симметричность
def mirror(in_polys, t=1e-6):
    mirror_coords = [[[-x, y] for x, y in poly] for poly in in_polys]
    mir_polys = unary_union([Polygon(coords) for coords in mirror_coords])
    ins_polys = unary_union([Polygon(coords) for coords in in_polys])
    if abs(mir_polys.area - ins_polys.area) > t:
        print('Не совпадаю площади при отражении')
        return None
    else:
        area = mir_polys.area
    int_area = mir_polys.intersection(ins_polys).area
    result = False
    if abs(int_area - area) < t:
        result = True
    return result, mirror_coords

# Создаёт списки вариантов с разной этажностью
def get_hpoly(in_polys, out_poly, k, idx_name, mir_name):
    res_list = []
    n_houses = len(in_polys)
    if n_houses < 1:
        return None
    if n_houses == 1:
        hos_name = 11
        h = [(h_min + np.random.rand(1) * (h_max - h_min)).item()*k]
        name = f'{idx_name}_{mir_name}_{str(hos_name)}'
        res_list.append((name, in_polys, out_poly, k, h))
    elif n_houses == 2:
        hos_name = 21
        h1 = (h_min + np.random.rand(1) * (h_max - h_min)).item()*k
        h = [h1]*n_houses
        name = f'{idx_name}_{mir_name}_{str(hos_name)}'
        res_list.append((name, in_polys, out_poly, k, h))
        hos_name = 22
        h1 = (h_min + np.random.rand(1) * (h_max - h_min)).item()*k
        h2 = (h_min + np.random.rand(1) * (h_max - h_min)).item()*k
        h = [h1, h2]
        name = f'{idx_name}_{mir_name}_{str(hos_name)}'
        res_list.append((name, in_polys, out_poly, k, h))
    else:
        hos_name = 31
        h1 = (h_min + np.random.rand(1) * (h_max - h_min)).item()*k
        h = [h1]*n_houses
        name = f'{idx_name}_{mir_name}_{str(hos_name)}'
        res_list.append((name, in_polys, out_poly, k, h))
        hos_name = 32
        h = [(h_min + np.random.rand(1) * (h_max - h_min)).item()*k for i in range(n_houses)]
        name = f'{idx_name}_{mir_name}_{str(hos_name)}'
        res_list.append((name, in_polys, out_poly, k, h))
        hos_name = 33
        n_h = int(n_houses/2 + 0.5)
        th = [(h_min + np.random.rand(1) * (h_max - h_min)).item()*k for i in range(n_h)]*2
        h = random.sample(th, n_houses)
        name = f'{idx_name}_{mir_name}_{str(hos_name)}'
        res_list.append((name, in_polys, out_poly, k, h))
    return res_list

In [5]:
# Запись всех вариантов
# 0 name
# 1 in_polys
# 2 out_poly
# 3 k
# 4 h - список высота
res_list = []
for i in tqdm(range(q_len)):
    in_polys = data['ins'][i]
    out_poly = data['out'][i]
    in_polys, out_poly, k = normalize(in_polys, out_poly)
    b, mir_polys = mirror(in_polys)
    idx_name = ('000000'+str(i))[-5:]
    if b:
        res_list += get_hpoly(in_polys, out_poly, k, idx_name, 0)
    else:
        res_list += get_hpoly(in_polys, out_poly, k, idx_name, 1)
        res_list += get_hpoly(mir_polys, [[-x, y] for x, y in out_poly], k, idx_name, 2)

In [6]:
volume_data_path = ...

# Сохранение объемных сцен как json
with open(volume_data_path, "w") as f:
    json.dump(res_list, f, indent=4, ensure_ascii=False)

**Генерация видов внутри объемной сцены**

In [7]:
volume_data_path = ...

In [8]:
with open(volume_data_path, "r", encoding="utf-8") as f:
    res_list = json.load(f)

In [9]:
# Максимальный отступ от границ участка
main_buffer = 30000

# Минимальный отступ от границ зданий
build_buffer = 10000

# Количество точек для генерации
n = 20

# Радиус для фильтрации точек
min_dist = 10000

In [10]:
# Рассчёт угла между векторами
def signed_angle(v1, v2):
    x1, y1 = v1
    x2, y2 = v2
    dot = x1 * x2 + y1 * y2
    det = x1 * y2 - y1 * x2
    angle = math.atan2(det, dot)
    return math.degrees(angle)

# Получение вектора из точек
def vector(p1, p2):
    return (p2[0] - p1[0], p2[1] - p1[1])

def get_angle(main_segment, coords):
    point_A = Point(main_segment[0])
    if Point(coords[0]).distance(point_A) <= Point(coords[1]).distance(point_A):
        oriented_seg = (coords[0], coords[1])
    else:
        oriented_seg = (coords[1], coords[0])
    v_main = vector(*main_segment)
    v_other = vector(*oriented_seg)
    angle = signed_angle(v_main, v_other)
    if angle >= 0:
        angle -= 90
    else:
        angle += 90
    return -angle

def find_zero(poly):
    zero_points = []
    for point in poly:
        if point[2] == 0:
            zero_points.append(point[:2])
    return zero_points

def sort_vertex(f1, points):
    verh = []
    niz = []
    for p in points:
        if p[2] == 0:
            niz.append(p)
        else:
            verh.append(p)
    v0 = Point(f1[:2]).distance(Point(verh[0][:2]))
    v1 = Point(f1[:2]).distance(Point(verh[1][:2]))
    n0 = Point(f1[:2]).distance(Point(niz[0][:2]))
    n1 = Point(f1[:2]).distance(Point(niz[1][:2]))
    res = [niz[0], niz[1]]
    if  n1 < n0:
        res = [niz[1], niz[0]]
    if v1 < v0:
        res = [verh[1]] + res + [verh[0]]
    else:
        res = [verh[0]] + res + [verh[1]]
    return res

def world_to_display(pt3d, matrix, width=1024, height=1024):
        pt = list(pt3d) + [1.0]
        transformed = [0.0] * 4
        matrix.MultiplyPoint(pt, transformed)
        if transformed[3] == 0.0:
            return None
        x_ndc = transformed[0] / transformed[3]
        y_ndc = transformed[1] / transformed[3]
        x_pix = int((x_ndc + 1) * 0.5 * width)
        y_pix = int((1 - y_ndc) * 0.5 * height)
        return (x_pix, y_pix)

# Создаёт объект сцены для VTK
def create_geometry(polygons_data, colors_list):
    points = vtk.vtkPoints()
    polygons = vtk.vtkCellArray()
    colors = vtk.vtkUnsignedCharArray()
    colors.SetNumberOfComponents(3)
    point_index = 0
    for i, polygon in enumerate(polygons_data):
        cell_point_ids = []
        for point in polygon:
            points.InsertNextPoint(point)
            cell_point_ids.append(point_index)
            point_index += 1
        polygons.InsertNextCell(len(cell_point_ids), cell_point_ids)
        colors.InsertNextTypedTuple(colors_list[i])
    polyData = vtk.vtkPolyData()
    polyData.SetPoints(points)
    polyData.SetPolys(polygons)
    polyData.GetCellData().SetScalars(colors)
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(polyData)
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    return actor

# Отображет 3D сцену
def plot_3d(polygon_data, colors_list):
    renderer = vtk.vtkRenderer()
    renderer.SetBackground(1, 1, 1)
    actor = create_geometry(polygon_data[1:], colors_list[1:])
    renderer.AddActor(actor)
    renderer.SetBackground(0, 0, 0)

    render_window = vtk.vtkRenderWindow()
    render_window.SetWindowName("3D Модель стены")
    render_window.SetSize(800, 600)
    render_window.AddRenderer(renderer)

    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    render_window.Render()
    render_window_interactor.Start()

# Генерация карты глубины
def get_deep(polygon_data, colors_list, camera_position, focal_point, view_angle, view_up=(0, 0, 1)):
    renderer = vtk.vtkRenderer()
    actor = create_geometry(polygon_data, colors_list)
    actor.GetProperty().LightingOff()
    actor.GetProperty().SetInterpolationToFlat()
    renderer.AddActor(actor)
    renderer.SetBackground(0, 0, 0)
    actor.GetProperty().SetAmbient(1.0)
    actor.GetProperty().SetDiffuse(0.0)
    actor.GetProperty().SetSpecular(0.0)


    render_window = vtk.vtkRenderWindow()
    render_window.SetOffScreenRendering(1)
    render_window.SetSize(1024, 1024)
    render_window.AddRenderer(renderer)

    # Настройка камеры
    camera = vtk.vtkCamera()
    camera.SetPosition(camera_position)
    camera.SetFocalPoint(focal_point)
    camera.SetViewUp(view_up)
    camera.SetViewAngle(view_angle)
    camera.SetClippingRange(1., 100000000.0)
    renderer.SetActiveCamera(camera)
    #renderer.ResetCameraClippingRange()
    render_window.Render()

    window_size = render_window.GetSize()
    z_buffer = vtk.vtkFloatArray()
    render_window.GetZbufferData(0, 0, window_size[0] - 1, window_size[1] - 1, z_buffer)
    z_buffer_np = vtk_np.vtk_to_numpy(z_buffer)
    z_buffer_np = z_buffer_np.reshape(window_size[1], window_size[0])
    z_buffer_np = 1 - z_buffer_np

    w2if = vtk.vtkWindowToImageFilter()
    w2if.SetInput(render_window)
    w2if.ReadFrontBufferOff()
    w2if.Update()
    vtk_image = w2if.GetOutput()
    width, height, _ = vtk_image.GetDimensions()
    vtk_array = vtk_image.GetPointData().GetScalars()
    np_image = vtk_to_numpy(vtk_array)
    np_image = np_image.reshape((height, width, -1))

    width, height = render_window.GetSize()
    aspect = width / height
    projection_matrix = camera.GetProjectionTransformMatrix(aspect, *camera.GetClippingRange())
    view_matrix = camera.GetViewTransformMatrix()
    total_transform = vtk.vtkMatrix4x4()
    vtk.vtkMatrix4x4.Multiply4x4(projection_matrix, view_matrix, total_transform)

    return z_buffer_np[::-1], np_image[::-1], total_transform

# Функция для создание палитры цветов
def get_colors(n_poly):
    R_dim = []
    d = 1/(n_poly+1)
    for i in range(1, n_poly+1):
        R_dim += [i*d]
    final = []
    for r in R_dim:
        r1 = [r, 0, 0]
        r2 = [r, 0, 1]
        r3 = [r, 1, 0]
        r4 = [r, 1, 1]
        final += [[r1,r2,r3,r4]]
    return np.round(np.array(final), 3)

def get_segments(polygons, k=1):
    all_segments = []
    for p in polygons:
        poly = list(p.exterior.coords)
        for i in range(len(poly)):
            start = [j/k for j in poly[i]]
            end = [j/k for j in poly[(i + 1) % len(poly)]]
            all_segments.append(LineString([start, end]))
    return all_segments

def generate_points_in_polygon(polygon, n):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < n:
        x = np.random.uniform(minx, maxx)
        y = np.random.uniform(miny, maxy)
        point = Point([(x, y)])
        if polygon.contains(point):
            points.append((x, y))
    return points

def filter_points(points, min_dist):
    points = np.array(points)
    tree = KDTree(points)
    filtered_points = []
    visited = set()
    for i, point in enumerate(points):
        if i in visited:
            continue
        neighbors = tree.query_ball_point(point, min_dist)
        visited.update(neighbors)
        filtered_points.append(point)
    filtered_points = np.array(filtered_points)
    return filtered_points

def extend_line(line: LineString, length) -> LineString:
    if len(line.coords) < 2:
        raise ValueError("Линия должна содержать хотя бы две точки")
    x1, y1 = line.coords[0]
    x2, y2 = line.coords[1]
    dx, dy = x2 - x1, y2 - y1
    norm = (dx**2 + dy**2) ** 0.5
    if norm == 0:
        raise ValueError("Первая и вторая точки линии совпадают")
    dx, dy = dx/norm, dy/norm
    x_new, y_new = x1 + dx * length, y1 + dy * length
    return LineString([(x1, y1), (x_new, y_new)])
    
def trim_line_at_polygon(line: LineString, polygons: list[Polygon]) -> LineString:
    min_distance = float("inf")
    nearest_point = None
    for poly in polygons:
        intersection = line.intersection(poly)
        if intersection.is_empty or intersection.geom_type == "MultiLineString":
            continue
        if intersection.geom_type == "LineString" :
            point = Point(intersection.coords[0])
        elif intersection.geom_type == "MultiPoint":
            point = min(intersection.geoms, key=lambda p: line.project(p))
        else:
            point = intersection
        if point.geom_type != "Point" :
            continue
        dist = line.project(point)
        if dist < min_distance:
            min_distance = dist
            nearest_point = point
    if nearest_point:
        return LineString([line.coords[0], nearest_point.coords[0]])
    return line

def get_lines(points, in_ps):
    lines = []
    angles = []
    for point in points:
        lines_point = []
        for i, pgl in enumerate(in_ps):
            pt = Point(point)
            for pgl_point in list(pgl.exterior.coords):
                line = LineString([pt, pgl_point])
                status = True
                for j, pgl2 in enumerate(in_ps):
                    inters = pgl2.intersection(line)
                    if not (type(inters) is Point or inters.is_empty):
                        status = False
                        break
                if status:
                    r1 = round(+50*np.random.rand(1).item())
                    r2 = round(-50*np.random.rand(1).item())
                    r1_line = rotate(line, r1, origin=line.coords[0], use_radians=False)
                    r2_line = rotate(line, r2, origin=line.coords[0], use_radians=False)
                    for li in [r1_line, r2_line]:
                        li_ex = extend_line(li, 1000000)
                        for pgl3 in in_ps:
                            if pgl3.intersects(li_ex):
                                lines_point.append(li_ex)
                                break
        lines += random.sample(lines_point, min([3, len(lines_point)]))
    for i in range(len(lines)):
        lines[i] = trim_line_at_polygon(lines[i], in_ps)
    return lines

def get_geometry(in_ps, out_p, h):
    pal = get_colors(len(in_ps))
    rectangles = [[]]
    for p_m in list(out_p.buffer(10000000, cap_style=2, join_style=2).exterior.coords):
        rectangles[0].append([*p_m, 0])
    colors_list = [(255, 255, 255)]
    for i in range(len(in_ps)):
        poly_cors = list(in_ps[i].exterior.coords)
        for j in range(4):
            p1 = [*poly_cors[j], 0]
            p2 = [*poly_cors[j+1], 0]
            p3 = [*poly_cors[j+1], h[i]]
            p4 = [*poly_cors[j], h[i]]
            poly_rec = [p1, p2, p3, p4]
            rectangles += [poly_rec]
            color = tuple(np.round(pal[i, j]*255).astype(np.uint8))
            colors_list += [color]
    return rectangles, colors_list

def get_focals(h, line, dist, t_bounds=(30,60)):
    focal1 = [*line.coords[0], h]
    razn = line.length - dist
    tg = razn/h
    alfa = np.degrees(np.arctan(tg))
    teta = round(t_bounds[0] + np.random.rand(1).item()*(t_bounds[1]-t_bounds[0]))/2
    beta = alfa+teta-90
    z = h + np.tan(np.radians(beta))*line.length
    focal2 = [*line.coords[1], z]
    return focal1, focal2, teta*2

def get_mask(image, colors_list, n=3):
    structure = generate_binary_structure(2, 1)
    poly_mask = [False]
    real_colors = []
    real_mask = []
    for color in colors_list[1:]:
        color_mask = np.all(image == color, axis=-1)
        erosion_mask = binary_erosion(color_mask, structure=structure, iterations=n)
        final_mask = binary_dilation(erosion_mask, structure=structure, iterations=n)
        if final_mask.sum() > 0:
            poly_mask.append(True)
            real_mask.append(final_mask)
            real_colors.append(np.round(np.array(color)/255, 3))
        else:
            poly_mask.append(False)
    return poly_mask, real_mask, real_colors

def unit_vector(point1, point2):
    point1 = np.array(point1)
    point2 = np.array(point2)
    vector = point2 - point1
    unit_vector = vector / np.linalg.norm(vector)
    return unit_vector

def get_view(name_scene, in_ps, out_p, k, h, path):
    in_ps = [[[x/k, y/k] for x, y in poly] for poly in in_ps]
    out_p = [[x/k, y/k] for x, y in out_p]
    h = [z/k for z in h]
    out_p = Polygon(out_p)
    main_polygon = out_p.buffer(main_buffer, cap_style=2, join_style=2)
    in_ps = [Polygon(i) for i in in_ps]
    holes = [i.buffer(build_buffer, cap_style=2, join_style=2) for i in in_ps]
    result_polygon = main_polygon
    for hole in holes:
        result_polygon = result_polygon.difference(hole)
    points = generate_points_in_polygon(result_polygon, n)
    points = filter_points(points, min_dist)
    lines = get_lines(points, in_ps)
    polygons_data, colors_list = get_geometry(in_ps, out_p, h)
    pal = get_colors(len(in_ps))
    views = []
    for view_id, line in enumerate(lines):
        f1, f2, teta = get_focals(1750, line, 10000)
        v = unit_vector(f1, f2).tolist()
        o = (np.array(f1)*k).tolist()
        deep, image, total_transform = get_deep(polygons_data, colors_list, f1, f2, teta)
        deep_name = f'{name_scene}_{view_id}_depth.jpg'
        image_name = f'{name_scene}_{view_id}_img.jpg'
        pil_img = Image.fromarray(image)
        pil_img.save(path+'/'+image_name)
        deep_norm = (deep- deep.min()) / (deep.ptp())
        deep_uint8 = (deep_norm * 255).astype(np.uint8)
        deep_rgb = np.stack([deep_uint8]*3, axis=-1)
        pil_deep = Image.fromarray(deep_rgb)
        pil_deep.save(path+'/'+deep_name)
        poly_mask, real_mask, real_colors = get_mask(image, colors_list)
        houses = {}
        s = 0
        for j, label in enumerate(poly_mask):
            if not label:
                continue
            else:
                diff = pal.astype(np.int32) - real_colors[s].astype(np.int32)
                dist_sq = np.sum(diff**2, axis=-1)
                min_coord_flat = np.argmin(dist_sq)
                h, f = np.unravel_index(min_coord_flat, dist_sq.shape)
                if h not in houses.keys():
                    angle = get_angle([f1[:2], f2[:2]], find_zero(polygons_data[j]))
                    dn = 'right' if angle < 0 else 'left'
                    sv = sort_vertex(f1, polygons_data[j])
                    pv = [world_to_display(p, total_transform) for p in sv]
                    mask_name = f'{name_scene}_{view_id}_{h}_{dn}.png'
                    s_mask = real_mask[s].astype(np.uint8) * 255
                    mask_rgb = np.stack([s_mask]*3, axis=-1)
                    pil_mask = Image.fromarray(mask_rgb)
                    pil_mask.save(path+'/'+mask_name)
                    houses[int(h)] = {dn: (real_colors[s].tolist(), pv, angle)}
                else:
                    angle = get_angle([f1[:2], f2[:2]], find_zero(polygons_data[j]))
                    dn = 'right' if angle < 0 else 'left'
                    sv = sort_vertex(f1, polygons_data[j])
                    pv = [world_to_display(p, total_transform) for p in sv]
                    mask_name = f'{name_scene}_{view_id}_{h}_{dn}.png'
                    s_mask = real_mask[s].astype(np.uint8) * 255
                    mask_rgb = np.stack([s_mask]*3, axis=-1)
                    pil_mask = Image.fromarray(mask_rgb)
                    pil_mask.save(path+'/'+mask_name)
                    houses[int(h)][dn] = (real_colors[s].tolist(), pv, angle)
                s += 1
        view_dict = {'o': o, 'v': v, 't': (teta-45)/10, 'houses': houses}
        json_name = f'{name_scene}_{view_id}.json'
        json_path = path+'/'+json_name
        with open(json_path, "w") as f:
            json.dump(view_dict, f, indent=4, ensure_ascii=False)

In [11]:
def record_views(scene_list, path):
    for scene in tqdm(scene_list):
        get_view(*scene, path)

# Запись всех итоговых видов
city_path = ...
record_views(res_list, city_path)

**Экстракция файлов**

Здесь для каждого вида определяется, сколько домов в них попало. Далее они сортируются так, чтобы в итоговом датасете было равное количество снимков с разным количеством домов.

In [12]:
city_path = r'E:city_data'

all_files = os.listdir(city_path)

view = {} # Маски каждого вида
for json_file in tqdm(all_files):
    if '.json' in json_file:
        name_v = json_file[:-5]
        views[name_v] = {}

for file in tqdm(all_files):
    name_v = None
    if '.json' in file or 'depth' in file or 'img' in file:
        continue
    if 'left' in file:
        pred_name = file.split('_left')[0]
        post = pred_name.split('_')[-1]
        name_v = pred_name[:-(1+len(post))]
        mask_name = post+'_left'
        views[name_v][mask_name] = file
    elif 'right' in file:
        pred_name = file.split('_right')[0]
        post = pred_name.split('_')[-1]
        name_v = pred_name[:-(1+len(post))]
        mask_name = post+'_right'
        views[name_v][mask_name] = file
    else:
        print('Странный файл!', file)
        break

with open('view_masks.json', 'w', encoding='utf-8') as file:
    json.dump(views, file, ensure_ascii=False, indent=4)

vd = {} # Виды по количеству домов
cd = {} # Количество видов на количество домов
for v in tqdm(views.keys()):
    with open(city_path+'/'+v+'.json', "r", encoding="utf-8") as f:
        img_info = json.load(f)
    hn = len(img_info['houses'].keys())
    if hn in cd.keys():
        vd[hn] += [v]
        cd[hn] += 1
    else:
        vd[hn] = [v]
        cd[hn] = 1

with open('view_counts.json', 'w', encoding='utf-8') as file:
    json.dump(vd, file, ensure_ascii=False, indent=4)

random.seed(42)
mod = cd[5]
total_num = 0
norm_vd = {}
for k in cd.keys():
    if k != 0:
        num_samples = min(cd[k], mod)
        total_num += num_samples
        random.shuffle(vd[k])

In [13]:
keys = sorted(list(cd.keys()))
if 0 in keys:
    keys = keys[1:]

v_list = [] # Итоговый список видов с равномерно распределенным количеством домов
for n in tqdm(range(cd[keys[0]])):
    for k in cd.keys():
        if n < cd[k] and k != 0:
            v_list += [vd[k][n]]

with open('sorted_views.json', 'w', encoding='utf-8') as file:
    json.dump(v_list, file, ensure_ascii=False, indent=4)